<a href="https://colab.research.google.com/github/safi50/Synthetic-Medical-Image-Generation-using-DCGANs/blob/main/Skin_Cancer_HAM10000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kmader_skin_cancer_mnist_ham10000_path = kagglehub.dataset_download('kmader/skin-cancer-mnist-ham10000')

print('Data source import complete.')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
import os
import shutil
import zipfile

# Adjust this path based on the actual dataset directory name
dataset_name = "skin-cancer-mnist-ham10000"  # Replace with the correct name if different
image_dir_part1 = f"/kaggle/input/{dataset_name}/HAM10000_images_part_1"
image_dir_part2 = f"/kaggle/input/{dataset_name}/HAM10000_images_part_2"
all_images_dir = '/kaggle/working/combined_images'

# Create a combined directory for all images
os.makedirs(all_images_dir, exist_ok=True)

# Combine images from both parts into a single directory
for part in [image_dir_part1, image_dir_part2]:
    for file in os.listdir(part):
        shutil.copy(os.path.join(part, file), os.path.join(all_images_dir, file))

print(f"Images combined into: {all_images_dir}")



In [ ]:
class HAM10000Dataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = [img for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_filename = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_filename)

        # Load the image
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image


In [ ]:
# Transformation function for images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1] for Tanh activation
])

# Load the dataset
train_dataset = HAM10000Dataset(image_dir='/kaggle/working/combined_images', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

print(f"Dataset loaded with {len(train_dataset)} images.")


In [ ]:
# Hyperparameters
latent_dim = 100           # Latent vector size
channels_img = 3           # Image channels (e.g., 3 for RGB)
lr = 0.0002                # Learning rate
betas = (0.5, 0.999)       # Beta values for Adam optimizer
num_epochs = 300           # Number of epochs
batch_size = 128            # Batch size
checkpoint_dir = "./checkpoints"  # Directory to save models

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Updated DCGAN Generator
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: z_dim x 1 x 1 (latent vector)
            nn.ConvTranspose2d(z_dim, 512, kernel_size=4, stride=1, padding=0),  # Output: (512, 4, 4)
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),  # Output: (256, 8, 8)
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # Output: (128, 16, 16)
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # Output: (64, 32, 32)
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, channels_img, kernel_size=4, stride=2, padding=1),  # Output: (channels_img, 64, 64)
            nn.Tanh()  # Tanh activation to get output in range [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)


# Updated DCGAN Discriminator
class Discriminator(nn.Module):
    def __init__(self, channels_img):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: (channels_img, 64, 64)
            nn.Conv2d(channels_img, 64, kernel_size=4, stride=2, padding=1),  # Output: (64, 32, 32)
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # Output: (128, 16, 16)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # Output: (256, 8, 8)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),  # Output: (512, 4, 4)
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0),  # Output: (1, 1, 1) - Real/Fake scalar
            nn.Sigmoid()  # Sigmoid activation for binary classification
        )

    def forward(self, x):
        return self.disc(x).view(-1)


# Initialize Generator and Discriminator
gen = Generator(latent_dim, channels_img).to(device)
disc = Discriminator(channels_img).to(device)

# Optimizers
optimizer_G = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCELoss()

# Fixed noise for generating images during training
fixed_noise = torch.randn(16, latent_dim, 1, 1).to(device)

# Training Loop
for epoch in range(num_epochs):
    for real_imgs in train_loader:
        real_imgs = real_imgs.to(device)  # Move real images to GPU
        current_batch_size = real_imgs.size(0)

        # Train Discriminator: Maximize log(D(x)) + log(1 - D(G(z)))
        optimizer_D.zero_grad()

        # Real images (label = 1)
        real_labels = torch.ones(current_batch_size, 1).to(device).squeeze()
        fake_labels = torch.zeros(current_batch_size, 1).to(device).squeeze()

        real_outputs = disc(real_imgs)
        real_loss = criterion(real_outputs, real_labels)

        # Fake images (label = 0)
        z = torch.randn(current_batch_size, latent_dim, 1, 1).to(device)  # Move noise to GPU
        fake_imgs = gen(z)
        fake_outputs = disc(fake_imgs.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator: Maximize log(D(G(z)))
        optimizer_G.zero_grad()

        fake_outputs = disc(fake_imgs)
        g_loss = criterion(fake_outputs, real_labels)  # Want generator to trick the discriminator

        g_loss.backward()
        optimizer_G.step()

    # Print losses and generate images every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

        # Save generated images
        with torch.no_grad():
            fake_imgs = gen(fixed_noise).detach().cpu()
            grid_img = utils.make_grid(fake_imgs, normalize=True)

            # Display images
            plt.figure(figsize=(10, 10))
            plt.title(f"Generated Images at Epoch {epoch+1}")
            plt.imshow(grid_img.permute(1, 2, 0))
            plt.axis('off')
            plt.show()

# Save final model checkpoints
torch.save(gen.state_dict(), os.path.join(checkpoint_dir, 'generator.pth'))
torch.save(disc.state_dict(), os.path.join(checkpoint_dir, 'discriminator.pth'))